
# Day 3b - Agent Memory (LangChain可本地运行版)
# 🧠 内存管理 - 第 2 部分 - 内存 (Memory)

欢迎来到 Kaggle 5 天 Agents 课程的第三天！

在上一篇 Notebook (Day 3a) 中，你学习了**会话 (Session)** 如何管理对话线程。
现在，你将添加 **“记忆” (Memory)** —— 这是一个可搜索的长期知识库，它可以在多个对话 (Sessions) 中持久保存。

**核心区别**:
*   **Session (会话)** = 短期记忆 (单次对话)
*   **Memory (记忆)** = 长期知识 (跨越多次对话)

---
**本 Notebook 将涵盖**:
1.  ✅ 初始化 MemoryService 并与 Agent 集成
2.  ✅ 将 Session 数据传输/摄取 (Ingest) 到 Memory 中
3.  ✅ 使用工具 (Tools) 搜索和检索记忆
4.  ✅ 自动化内存存储 (使用 Callbacks)
5.  ✅ 理解内存整合 (Memory Consolidation) 的概念

---
**原版**: Google ADK Team (Kaggle 5-Day Agent Course)  
**改写**: LangChain 实现
> **ℹ️ 声明**: 本教程改编自 Google [5-Day AI Agents Course](https://www.kaggle.com/learn-guide/5-day-agents)，由 LangChain 重写以适配本地运行。内容仅供学习研究。


## ⚙️ Section 1: Setup (环境设置)

### 1.1 切换内核
- 点击右上角的 Kernel (内核) 选择区域。
- 在列表中选择 Python (5-DAY-AGENTS)。

> 关于如何初始化环境和创建内核，请查看 [README 如何运行](../README.md#如何运行)

### 1.2 安装依赖

In [ ]:
# 优选 uv 包管理器 
!uv pip install langchain langchain-openai langchain-community python-dotenv


### 1.3 导入依赖库

In [ ]:
import os
import uuid
from typing import List, Dict, Any
from datetime import datetime

# LangChain Imports
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.tools import tool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.callbacks import BaseCallbackHandler

print("✅ 依赖库加载完成")

### 1.4 配置 API Key

In [ ]:
# 加载环境变量
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

# 配置模型
MODEL_NAME = os.environ.get("MODEL_NAME")
print(f"✅ 使用模型: {MODEL_NAME}")

# 定义全局 Session 存储
# Key: session_id, Value: ChatMessageHistory
session_store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in session_store:
        session_store[session_id] = ChatMessageHistory()
    return session_store[session_id]

print("✅ ADK components (simulation) imported successfully.")

### 1.5 辅助函数 (Async)

In [ ]:

# 使用异步版本，以匹配原教程的使用习惯 (await run_session)

async def run_session(
    agent_executor, 
    user_queries: List[str] | str, 
    session_id: str = "default"
):
    """Helper function to run queries in a session and display responses."""
    print(f"\n### Session: {session_id}")

    if isinstance(user_queries, str):
        user_queries = [user_queries]
    
    config = {"configurable": {"session_id": session_id}}
    
    for query in user_queries:
        print(f"\nUser > {query}")
        
        # 使用 ainvoke (异步调用)
        response = await agent_executor.ainvoke(
            {"input": query},
            config=config,
        )
        
        # 提取回复
        output = response.get("output", "")
        print(f"Model: > {output}")


### 1.6 配置重试 (Retry Options)

In [ ]:

# LangChain 的 ChatOpenAI 默认有重试机制，也可以通过 max_retries 参数配置。
llm = ChatOpenAI(model=MODEL_NAME, temperature=0.7, max_retries=5)
print("✅ Retry options configured via LangChain model.")



## 🧠 Section 3: Initialize MemoryService (初始化内存服务)

**3.1 初始化 Memory**

我们将实现一个 `InMemoryMemoryService` 类，模拟 ADK 的功能：
1.  **存储 (Store)**: 保存来自 Session 的对话记录。
2.  **搜索 (Search)**: 提供基于关键词的搜索功能。


In [ ]:

class InMemoryMemoryService:
    def __init__(self):
        # 存储记忆列表，每个记忆是一个字典
        self.memories = []

    def add_session_to_memory(self, session: ChatMessageHistory):
        """将 Session 中的对话保存到长期记忆中"""
        # 在这里，我们将整个 Session 转换为一段文本进行存储
        # 实际生产中 (如 Vertex AI) 会进行更复杂的提取和摘要
        conversation_text = ""
        for msg in session.messages:
            role = "User" if isinstance(msg, HumanMessage) else "Model"
            conversation_text += f"{role}: {msg.content}\n"
        
        if conversation_text.strip():
            self.memories.append({
                "content": conversation_text,
                "timestamp": datetime.now().isoformat(),
                "id": str(uuid.uuid4())
            })
            print(f"✅ Session stored. Total memories: {len(self.memories)}")

    def search_memory(self, query: str) -> List[str]:
        """基于关键词搜索记忆"""
        results = []
        # 简单的不区分大小写的关键词匹配
        query_terms = query.lower().split()
        
        for mem in self.memories:
            content = mem["content"]
            content_lower = content.lower()
            # 如果查询中的任意词出现在内容中
            if any(term in content_lower for term in query_terms):
                results.append(content)
        
        return results

# 初始化服务
memory_service = InMemoryMemoryService()
print("✅ InMemoryMemoryService initialized.")



## 3.2 Add Memory to Agent (为代理添加内存)

在 LangChain 中，要让 Agent 使用内存，并不像 ADK 那样直接传 `memory_service` 给 Runner。
我们需要：
1.  创建一个 Agent。
2.  (后续步骤) 将内存搜索功能封装为 **Tool** 给 Agent 使用。

首先，我们先创建一个**不带内存工具**的基础 Agent，用于产生一些 Session 数据。


In [ ]:

# 定义一个基础 Chain (无工具版本)
# 既然这一步只需要产生对话数据，不需要 Agent 的工具调用能力，我们可以直接使用 LLM Chain。
# 这样避免了为了创建 Agent 而必须提供 Dummy Tool 的尴尬。

prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer user questions in simple words."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
])

# 定义基础 Chain
chain = prompt | llm

# 适配器函数：将 AIMessage 输出转换为 run_session 期望的字典格式 {'output': ...}
def agent_style_output(ai_message):
    return {"output": ai_message.content}

# 组合 Chain
basic_chain = chain | agent_style_output

# Runner (RunnableWithMessageHistory)
basic_runner = RunnableWithMessageHistory(
    basic_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

print("✅ Basic Runner created (Pure LLM Chain, no Tools/Agent needed).")



## 💾 Section 4: Ingest Session Data into Memory (将会话数据导入内存)

现在我们有了 Memory Service 和 Session Service (runnable with history)，但它们是分离的。
我们需要手动将会话数据“摄取”到内存中。

首先，让我们产生一段对话。


In [ ]:

# 运行一段对话
await run_session(
    basic_runner,
    "My favorite color is blue-green. Can you write a Haiku about it?",
    session_id="conversation-01"
)


In [ ]:

# 验证 Session 内容
session = get_session_history("conversation-01")
print("📝 Session contains:")
for msg in session.messages:
    role = "user" if isinstance(msg, HumanMessage) else "model"
    print(f"  {role}: {msg.content[:60]}...")


In [ ]:

# 关键步骤！手动将 Session 添加到 Memory
# 在 ADK 中: await memory_service.add_session_to_memory(session)

memory_service.add_session_to_memory(session)

print("✅ Session added to memory!")



## 🔎 Section 5: Enable Memory Retrieval in Your Agent (在代理中启用内存检索)

现在数据在 Memory 里了，我们需要给 Agent 一把“钥匙”来访问它。
这把钥匙就是 **Tool**。

我们将定义两个 Tool:
1.  `load_memory` (Reactive/被动): Agent 决定何时搜索。
2.  `preload_memory` (Proactive/主动): 总是搜索 (将在 Section 6 中模拟)。

### 5.2 Add Load Memory Tool


In [ ]:

@tool
def load_memory(query: str) -> str:
    """
    Useful whenever you need to recall past conversations or information about the user, 
    preferences, or specific facts mentioned effectively in previous chats.
    Do NOT use this for the current conversation context.
    """
    print(f"\n  [Tool Call] Searching memory for: '{query}'")
    results = memory_service.search_memory(query)
    if not results:
        return "No relevant memories found."
    return f"Found {len(results)} relevant memories:\n" + "\n---\n".join(results)

print("✅ `load_memory` tool defined.")


In [ ]:

# 创建带有 memory tool 的 Agent
tools_with_memory = [load_memory]

# 更新 Prompt，明确告知可以通过工具查阅记忆
prompt_with_memory = ChatPromptTemplate.from_messages([
    ("system", "Answer user questions in simple words. Use load_memory tool if you need to recall past conversations."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

agent_with_memory = create_tool_calling_agent(llm, tools_with_memory, prompt_with_memory)

agent_runner_memory = RunnableWithMessageHistory(
    AgentExecutor(agent=agent_with_memory, tools=tools_with_memory, verbose=False),
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

print("✅ Agent with load_memory tool created.")


In [ ]:

# 测试 1: 在一个新的 Session 中询问之前存储的信息 (Favorite Color)
await run_session(agent_runner_memory, "What is my favorite color?", "color-test")



### 5.4 Complete Manual Workflow Test (完整手动工作流测试)

1.  对话 (About Birthday)
2.  手动保存
3.  新对话检索


In [ ]:

await run_session(agent_runner_memory, "My birthday is on March 15th.", "birthday-session-01")


In [ ]:

# 手动保存
birthday_session = get_session_history("birthday-session-01")
memory_service.add_session_to_memory(birthday_session)
print("✅ Birthday session saved to memory!")


In [ ]:

# 在新 Session 中测试回忆
await run_session(agent_runner_memory, "When is my birthday?", "birthday-session-02")



### 🚀 Proactive vs Reactive Memory (主动式 vs 响应式内存)

在原来的 ADK 教程中，介绍了两种主要的内存使用模式：`load_memory` (Reactive) 和 `preload_memory` (Proactive)。

**1. Reactive (响应式) - `load_memory`**
*   **机制**: 赋予 Agent 一个工具 (如我们上面定义的 `load_memory`)。
*   **行为**: Agent 根据对话上下文**自主决定**何时搜索记忆。就像我们在 Section 5.2 中做的那样。
*   **优点**: 效率高，节省 Token（不需要时就不搜索）。
*   **缺点**: Agent 可能会“忘记”去搜索，或者判断失误。

**2. Proactive (主动式) - `preload_memory`**
*   **机制**: 在每一轮对话开始**之前**，系统自动搜索相关记忆并注入到 Context 中。
*   **行为**: 无论 Agent 是否请求，检索到的记忆总是直接呈现在 Prompt 中。
*   **优点**: 保证上下文总是存在的，Agent 不需要“思考”去搜索，降低了 Agent 的推理负担。
*   **缺点**: 可能会消耗更多 Token（即使是不相关的查询也会搜索并注入上下文）。

💡 **LangChain 实现注记**: 
在 LangChain 中，"Proactive Memory" 通常通过 **RAG (检索增强生成)** 模式来实现：即在 Chain 的第一步先运行 Retrieval，然后将结果填充到 Prompt 的 Template 中，最后再传给 LLM。

为了保持教程简洁，本 Notebook 重点演示了 Agentic (Reactive) 的方式，但理解这两种模式的区别对于设计复杂的 AI 应用至关重要。

---


### 5.5 手动内存搜索 (Manual Search)

In [ ]:

search_results = memory_service.search_memory("favorite color")
print("🔍 Search Results:")
for res in search_results:
    print(f"  Found: {res[:200]}...")



## 🤖 Section 6: Automating Memory Storage (自动化内存存储)

目前为止，我们都是手动调用 `add_session_to_memory` 将会话保存到长期记忆中。在生产环境中，这应该自动发生。

在 Google ADK 中，这通过 **Callbacks (回调)** 机制实现 (`after_agent_callback`)。
LangChain 也拥有强大的 Callbacks 系统 (`BaseCallbackHandler`)，但为了保持本教程的简单直观，我们将通过一个**包装函数 (Wrapper Function)** 来模拟这一行为。

这个包装函数 `run_session_with_autosave` 将在每次会话结束后，自动提取并保存对应的 Session 数据。


In [ ]:

# 此逻辑模拟 after_agent_callback
async def run_session_with_autosave(agent_executor, query, session_id):
    # 1. 运行正常的 Session
    await run_session(agent_executor, query, session_id=session_id) 
    
    # 2. 运行结束后自动保存 Logic
    print(f"[Auto-Save] Saving session {session_id}...")
    session = get_session_history(session_id)
    memory_service.add_session_to_memory(session)

print("✅ Auto-save logic defined.")


### 6.3 & 6.4 Test Automatic Memory

In [ ]:

# Test 1: Tell about a new gift
# 注意：我们这里重用了 agent_runner_memory (带有 load_memory 工具)
await run_session_with_autosave(
    agent_runner_memory, 
    "I gifted a new toy to my nephew on his 1st birthday!", 
    "auto-save-test"
)

# Test 2: New session, ask about gift
# Agent should use tool to find it
await run_session(
    agent_runner_memory,
    "What did I gift my nephew?",
    "auto-save-test-2"
)



## 🧩 Section 7: Memory Consolidation (记忆巩固)

### 7.1 The Limitation of Raw Storage (原始存储的局限性)
目前的 Memory 实现是将所有对话原始数据直接存储。随着对话增多，这会导致：
*   存储空间浪费
*   检索结果包含大量噪音
*   Token 消耗过大

### 7.2 What is Memory Consolidation? (什么是记忆巩固？)
记忆巩固 = 提取重要事实，丢弃对话噪音。

**Before (Raw Storage):**
> User: "我最喜欢的颜色是蓝绿色。我也喜欢紫色。其实大部分时间我还是喜欢蓝绿色。"
> Agent: "好的！我会记住的。"

**After (Consolidation):**
> Extracted Memory: "User's favorite color: BlueGreen"

### 7.3 How Consolidation Works (运作原理)
在 Vertex AI Memory Bank 等高级服务中，这通常是自动发生的：
1.  **Input**: 原始 Session Events
2.  **Process**: LLM 分析对话，提取关键事实 (Facts, Entities)
3.  **Output**: 结构化的、简洁的记忆条目

在本 Notebook 中，我们虽然只实现了基础的存储，但概念是通用的：**长期记忆应该是高度提炼的知识，而非流水账。**

在 Day 5 中，你将深入了解 Vertex AI 是如何处理这一点的。



## 📊 Summary (总结)

恭喜！你已经学会了如何在 LangChain 中模拟 ADK 的 Memory 机制：

1.  **MemoryService**: 创建了一个存储和检索对话记录的服务。
2.  **Tools**: 赋予 Agent `load_memory` 工具，使其能够自主查阅长期记忆。
3.  **Ingestion**: 实现了将会话 (Short-term) 保存到 记忆 (Long-term) 的流程。
4.  **Automation**: 了解了如何通过 AUTOMATION 自动触发保存。

**Next**: Day 4 将介绍可观测性 (Observability) 和评估 (Evaluation)。
